In [2]:
import torch
import torch.nn as nn
import pandas as pd 
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
import torchvision.models as models
import torchaudio
import torchaudio.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import torchaudio.transforms as T
import random

In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import torch
import numpy as np
import tensorflow_hub as hub

In [5]:
Id = []

for dirname, _, filenames in os.walk('/kaggle/input/environmental-sound-classification-50/audio/audio/16000'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
    break
    
train_paths, test_paths = train_test_split(Id, test_size=0.2, random_state=42)
Id[:5]

['/kaggle/input/environmental-sound-classification-50/audio/audio/16000/5-257349-A-15.wav',
 '/kaggle/input/environmental-sound-classification-50/audio/audio/16000/5-195557-A-19.wav',
 '/kaggle/input/environmental-sound-classification-50/audio/audio/16000/2-122820-B-36.wav',
 '/kaggle/input/environmental-sound-classification-50/audio/audio/16000/1-115920-A-22.wav',
 '/kaggle/input/environmental-sound-classification-50/audio/audio/16000/1-172649-C-40.wav']

In [ ]:
# inputs = {ort_session.get_inputs()[0].name: w}
# outputs = ort_session.run(None, inputs)

# # Get the 3 outputs
# class_scores = outputs[0]  # Class predictions (521 classes)
# embeddings = outputs[1]    # Embeddings (feature vector)
# logit_predictions = outputs[2]  # Logits

In [6]:
train = pd.DataFrame({'filename':train_paths })
train.head()

,filename
0,/kaggle/input/environmental-sound-classificati...
1,/kaggle/input/environmental-sound-classificati...
2,/kaggle/input/environmental-sound-classificati...
3,/kaggle/input/environmental-sound-classificati...
4,/kaggle/input/environmental-sound-classificati...


In [7]:
len(train['filename'])

1600

In [ ]:
# for signature in yamnet_model.signatures:
#     print(f"Signature: {signature}")
#     print(yamnet_model.signatures[signature].structured_outputs)


In [8]:
train['label'] = train['filename'].apply(lambda x: int(os.path.basename(x).split('-')[-1].split('.')[0]))
print(train[['filename', 'label']].head())


                                            filename  label
0  /kaggle/input/environmental-sound-classificati...      2
1  /kaggle/input/environmental-sound-classificati...     34
2  /kaggle/input/environmental-sound-classificati...     44
3  /kaggle/input/environmental-sound-classificati...     20
4  /kaggle/input/environmental-sound-classificati...     22


In [9]:
test = pd.DataFrame({'filename':test_paths })
test['label'] = test['filename'].apply(lambda x: int(os.path.basename(x).split('-')[-1].split('.')[0]))
print(test[['filename', 'label']].head())


                                            filename  label
0  /kaggle/input/environmental-sound-classificati...     29
1  /kaggle/input/environmental-sound-classificati...     37
2  /kaggle/input/environmental-sound-classificati...      8
3  /kaggle/input/environmental-sound-classificati...     47
4  /kaggle/input/environmental-sound-classificati...      7


In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(yamnet_model.parameters(), lr=1e-4)


In [ ]:
# Augmentation

# def add_white_noise( x, noise_percentage_factor):
#     noise = np.random.normal(0, x.std(), x.size)
#     return x + noise * noise_percentage_factor

# # def stime_stretch( x, time_stretch_rate):
# #         return librosa.effects.time_stretch(x, time_stretch_rate)
# # def add_time_stretch(x, time_stretch_rate):
# #     return librosa.effects.time_stretch(x, time_stretch_rate)
# def pitch_scale( x, sr, num_semitones):
#     return librosa.effects.pitch_shift(x, sr, num_semitones)


In [10]:
df = pd.read_csv('/kaggle/input/environmental-sound-classification-50/esc50.csv')

pd.set_option('display.max_columns', None)

print(df.head())  

            filename  fold  target        category  esc10  src_file take
0   1-100032-A-0.wav     1       0             dog   True    100032    A
1  1-100038-A-14.wav     1      14  chirping_birds  False    100038    A
2  1-100210-A-36.wav     1      36  vacuum_cleaner  False    100210    A
3  1-100210-B-36.wav     1      36  vacuum_cleaner  False    100210    B
4  1-101296-A-19.wav     1      19    thunderstorm  False    101296    A


In [11]:
classes = dict(zip(df['target'], df['category']))

print(classes)


{0: 'dog', 14: 'chirping_birds', 36: 'vacuum_cleaner', 19: 'thunderstorm', 30: 'door_wood_knock', 34: 'can_opening', 9: 'crow', 22: 'clapping', 48: 'fireworks', 41: 'chainsaw', 47: 'airplane', 31: 'mouse_click', 17: 'pouring_water', 45: 'train', 8: 'sheep', 15: 'water_drops', 46: 'church_bells', 37: 'clock_alarm', 32: 'keyboard_typing', 16: 'wind', 25: 'footsteps', 4: 'frog', 3: 'cow', 27: 'brushing_teeth', 43: 'car_horn', 12: 'crackling_fire', 40: 'helicopter', 29: 'drinking_sipping', 10: 'rain', 7: 'insects', 26: 'laughing', 6: 'hen', 44: 'engine', 23: 'breathing', 20: 'crying_baby', 49: 'hand_saw', 24: 'coughing', 39: 'glass_breaking', 28: 'snoring', 18: 'toilet_flush', 2: 'pig', 35: 'washing_machine', 38: 'clock_tick', 21: 'sneezing', 1: 'rooster', 11: 'sea_waves', 42: 'siren', 5: 'cat', 33: 'door_wood_creaks', 13: 'crickets'}


In [ ]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter

# from helper import _plot_signal_and_augmented_signal

# install pydub for using HighPassFilter
# install audiomentations

# Raw audio augmentation
augment_raw_audio = Compose(
    [
#         AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.015, p=1),
        PitchShift(min_semitones=-8, max_semitones=8, p=1),
        HighPassFilter(min_cutoff_freq=2000, max_cutoff_freq=4000, p=1)
    ]
)

In [12]:
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

In [13]:
!pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.2 MB/s eta 0:00:00a 0:00:01


In [14]:
!pip install torch onnx onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 82.6 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [15]:
!mkdir -p saved_model


In [16]:
concrete_func = yamnet_model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

# Save the model with the concrete function
tf.saved_model.save(yamnet_model, 'saved_model/yamnet_model', signatures=concrete_func)

In [17]:
!python -m tf2onnx.convert --saved-model saved_model/yamnet_model --output yamnet.onnx


/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-10-18 22:52:42,464 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-10-18 22:52:47,783 - INFO - Signatures found in model: [serving_default].
2024-10-18 22:52:47,783 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-10-18 22:52:47,784 - INFO - Output names: ['output_0', 'output_1', 'output_2']
2024-10-18 22:52:49,620 - INFO - Using tensorflow=2.16.1, onnx=1.17.0, tf2onnx=1.16.1/15c810
2024-10-18 22:52:49,621 - INFO - Using opset <onnx, 15>
2024-10-18 22:52:49,797 - INFO - Computed 0 values for constant folding
2024-10-18 22:52:50,257 - INFO - Optimizing ONNX model
2024-10-18 22:52:51,947 - INFO - After optimization: BatchNormalization -27 (27->0), Cast -8 (19->11), Concat -3 (

In [18]:
import onnxruntime as ort
import numpy as np
import librosa

# Load ONNX model
ort_session = ort.InferenceSession('yamnet.onnx')


In [19]:
def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=16000):
    """Resample waveform if required using PyTorch."""
    if original_sample_rate != desired_sample_rate:
        resample_transform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=desired_sample_rate)
        waveform = resample_transform(waveform)
    return desired_sample_rate, waveform # sr and signal


In [20]:
def extract_embeddings(filename):
    y, sr = librosa.load(filename, sr=22050)

    s,w=ensure_sample_rate(sr, y, desired_sample_rate=22050)
    inputs = {ort_session.get_inputs()[0].name: w}
    outputs = ort_session.run(None, inputs)
    class_scores = outputs[0]  # Class predictions (521 classes)
    embeddings = outputs[1]    # Embeddings (feature vector)
    logit_predictions = outputs[2]  # Logits
    return class_scores, embeddings
all_embeddings = []
all_labels = []

In [21]:
for i in range(len(train)):
        
        filename, label = train['filename'][i],  train['label'][i]
        c,e=extract_embeddings(filename)
        all_embeddings.append(e)
        all_labels.append(label)
all_embeddings = np.array(all_embeddings)  # Shape: (num_samples, 1024)
all_labels = np.array(all_labels)

In [ ]:
# for epoch in range(5):
#     avg_loss, accuracy = train(model, criterion, optimizer, train_loader,device )
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple fully connected classifier in PyTorch
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

# Instantiate the classifier for ESC-50 (50 classes)
input_dim = 1024  # YAMNet embeddings are 1024-D
num_classes = 50  # ESC-50 has 50 classes
model = Classifier(input_dim, num_classes)


In [23]:
from sklearn.model_selection import train_test_split

# Split embeddings into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(all_embeddings, all_labels, test_size=0.2, random_state=42)


In [24]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    outputs = outputs.mean(dim=1)  # Shape becomes [1280, 50]
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


epoch:  0 loss:  1.0146068847465515 accuracy:  46.704 %
---------------------------
epoch:  1 loss:  0.7963647353076935 accuracy:  52.76599999999999 %
---------------------------
epoch:  2 loss:  0.7743401646614074 accuracy:  53.76 %
 saved at /kaggle/working/cnnnet_epoch_3.pth
---------------------------
epoch:  3 loss:  0.7466917690753937 accuracy:  54.276 %
---------------------------
epoch:  5 loss:  0.7228994320201874 accuracy:  55.234 %
 saved at /kaggle/working/cnnnet_epoch_6.pth
---------------------------
epoch:  6 loss:  0.7195748014545441 accuracy:  55.468 %
---------------------------
epoch:  7 loss:  0.7127152788257599 accuracy:  55.554 %
---------------------------
epoch:  8 loss:  0.7048812763404846 accuracy:  55.928 %
 saved at /kaggle/working/cnnnet_epoch_9.pth
---------------------------
epoch:  9 loss:  0.6917364977455139 accuracy:  56.266 %
---------------------------
epoch:  10 loss:  0.6951166493034363 accuracy:  56.294 %
---------------------------
epoch:  11 lo

In [28]:
# Test

with torch.no_grad():
    model.eval()
    total_loss=0.0 
    total_correct=0.0 
    for inputs, labels in testloader:
        labels=labels.to(device)
        outputs=model(inputs.to(device))
        loss=criterion(outputs, labels)
        total_loss+=loss.item()*inputs.size(0)
        output_idx=torch.argmax(outputs, dim=1)
        total_correct+=sum(labels==output_idx).sum().item() 
        accuracy = (total_correct / test_len) * 100
        loss = total_loss / test_len

    print(f'Accuracy: {accuracy:.2f}% Loss: {loss:.4f}')


Accuracy: 72.06% Loss: 0.5240
